<a href="https://colab.research.google.com/github/gacerioni/redis-workshop-series-ptbr-gabs/blob/main/redis_workshop_search_and_query_nov_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop - Redis Search and Query Engine: JSON, Hashes, e Indexação Avançada


![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

**Bem-vindos a mais um Workshop!**

Aqui, vamos explorar o uso do **Redis Search and Query Engine** para transformar o Redis em um banco de dados NoSQL poderoso, ideal para consultas avançadas e armazenamento de documentos. Neste módulo, focaremos em como o Redis pode ir além do caching e atuar como um verdadeiro banco de dados de busca e consulta.

### Objetivos do Workshop

Este notebook é uma introdução hands-on ao uso do Redis como um banco de dados de documentos, com as seguintes funcionalidades:

- **JSON e Hashes**: Estruturas de dados ideais para documentos e modelos aninhados, permitindo leitura e escrita de partes específicas de documentos JSON e armazenamento eficiente de pares chave-valor.
- **Indexação e Busca Avançada com RediSearch**: Inclui suporte para consultas textuais, filtros numéricos, buscas por localização geográfica e correspondência semântica, entre outras funcionalidades.

### O que vamos cobrir:

1. **Armazenamento de Documentos JSON e Hashes**: Aprenderemos como salvar, acessar e atualizar partes específicas de documentos JSON.
2. **Busca e Filtros no RediSearch**: Como criar índices para permitir buscas por tags, intervalos numéricos, e localização geográfica.
3. **Consultas Avançadas**: Como combinar diferentes critérios de busca para consultas poderosas e dinâmicas.

Para aproveitar melhor o workshop, recomendamos utilizar o **Redis Insight** para visualizar os dados e realizar consultas de forma interativa.

---

## Setup Rápido - Configuração do Ambiente

Para começar, precisamos garantir que temos todas as dependências instaladas e a conexão configurada para o Redis Cloud. Vamos instalar a biblioteca `redis-py` para Python e a ferramenta `redis-cli` para realizar testes de linha de comando diretamente.

### Passo 1: Instalação das Dependências


In [ ]:
# Instalando a biblioteca redis-py para interação com o Redis
!pip install -q redis

# Instalando o redis-cli para comandos diretos no terminal
!apt-get update -y
!apt-get install -y redis-tools

### Passo 2: Configurando a Conexão com o Redis Cloud

No bloco abaixo, configure seu endpoint do Redis Cloud com `host`, `port`, e `password`.

Aqui, mostramos um exemplo de configuração. Substitua os valores pelos do seu próprio Redis Cloud para conectar. O Redis Cloud oferece TLS para uma conexão segura; neste exemplo, vamos desativá-lo para simplicidade, mas recomendamos o uso do TLS para produção.

In [ ]:
import os

# Configurações do Redis Cloud - ajuste os valores para seu Redis Cloud
REDIS_HOST = "redis-18443.c309.us-east-2-1.ec2.redns.redis-cloud.com"
REDIS_PORT = 18443
REDIS_PASSWORD = "secret42"

# Montando a URL de conexão
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
os.environ["REDIS_CONN"] = f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"

### Passo 3: Testando a Conexão com o Redis Cloud

Para garantir que estamos conectados corretamente, vamos realizar um teste com o comando `PING`. Se a conexão estiver configurada corretamente, o Redis retornará "PONG".

**Nota:** Também incluímos um comando `FLUSHDB` comentado, que apaga todos os dados da base. Use-o com cuidado!

In [ ]:
# Testando a conexão com o Redis CLI
!redis-cli $REDIS_CONN PING

# Opcional: para reiniciar completamente o banco de dados, descomente a linha abaixo
# !redis-cli $REDIS_CONN FLUSHDB

Se o comando `PING` retornar `PONG`, sua configuração está correta e você já está conectado ao Redis Cloud!

Com o ambiente pronto, vamos explorar o armazenamento de documentos com Redis JSON e Hashes, os "heróis" do Redis para dados estruturados e aninhados.

# Hashes e JSON - Estruturas de Dados para Documentos

No Redis, os tipos de dados **Hash** e **JSON** fornecem estruturas ideais para armazenar e manipular objetos complexos, como registros de usuários ou informações de produtos. Nesta seção, vamos explorar as principais operações e manipulações possíveis com esses tipos de dados.

Os **Hashes** no Redis são semelhantes a um dicionário em Python ou um mapa em Java, sendo uma coleção de pares campo-valor. Já o tipo de dado **JSON** permite armazenar objetos completos, aninhados, e realizar consultas avançadas diretamente na estrutura JSON, oferecendo flexibilidade para aplicações que lidam com dados hierárquicos.

**Importante:** Tanto os **Hashes** quanto o **JSON** são indexáveis e preparados para **Search and Query**, permitindo que o Redis funcione como um banco de dados NoSQL robusto.


Vamos iniciar explorando o tipo de dado Hash no Redis, usando exemplos práticos.

## Redis Hash: Estrutura e Exemplos Práticos

Hashes são ideais para representar objetos com múltiplos atributos, como o perfil de um usuário, com campos como nome, e-mail e idade. Eles oferecem operações rápidas de leitura e escrita, permitindo que cada campo dentro do hash seja acessado e modificado individualmente, sem alterar toda a estrutura.

### Exemplo Prático: Criando e Manipulando um Hash no Redis

Usaremos o `HSET` para salvar variáveis como campos em um hash no Redis. Para praticar, vamos criar o registro de um "MVP" do esporte.

In [ ]:
# Criação de um hash representando um atleta MVP (Most Valuable Player)
!redis-cli $REDIS_CONN HSET mvp:1 name "Ayrton" last_name "Senna" email "senninha@f1.io" age 34

# Exibindo o resultado do hash criado
!redis-cli $REDIS_CONN HGETALL mvp:1

Neste exemplo, criamos um hash com campos de informações de um atleta, incluindo nome, sobrenome, e-mail e idade.

### Exercício: Criar Outro Hash

Agora, crie outro MVP no Redis usando o mesmo comando `HSET`. Esse exemplo será de um jogador de futebol bem conhecido.

In [ ]:
# Criação de outro hash representando um segundo MVP
!redis-cli $REDIS_CONN HSET mvp:2 name "Edson" last_name "Arantes do Nascimento" apelido "Pelé" email "pele@santosfc.com" age 82

# Exibindo o resultado do segundo hash criado
!redis-cli $REDIS_CONN HGETALL mvp:2

### Manipulando Hashes no Redis

Podemos visualizar todos os campos e valores de um hash usando o comando `HGETALL`. Outros comandos úteis incluem `HMGET` para obter valores específicos, `HINCRBY` para incrementar valores numéricos, e `HDEL` para excluir campos.

Abaixo estão alguns exemplos práticos.

In [ ]:
# Configuração da conexão com o Redis (certifique-se de já ter conectado antes, ou use este bloco para conectar)
import redis

# Definindo a conexão com o Redis Cloud
r = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD,
    decode_responses=True
)

# Exemplo de recuperação de valores específicos usando HMGET
valores = r.hmget("mvp:2", "name", "apelido", "age")
print("Campos selecionados do segundo MVP:", valores)

# Incrementando o campo 'gols' para simular a contagem de gols
# Nota: O campo 'gols' será criado automaticamente se não existir
r.hincrby("mvp:2", "gols", 1000)  # Adiciona 1000 ao campo 'gols'
r.hincrby("mvp:2", "gols", 5)     # Incrementa mais 5 gols

# Visualizando o hash atualizado
print("Hash atualizado do segundo MVP:", r.hgetall("mvp:2"))

#### Exercício: Excluindo Campos de Hashes

Agora, vamos usar o comando `HDEL` para excluir o campo 'gols' do hash `mvp:2`.

In [ ]:
# Excluir o campo 'gols' do segundo MVP
r.hdel("mvp:2", "gols")

# Verificando o hash atualizado
print("Hash final do segundo MVP após a exclusão do campo 'gols':", r.hgetall("mvp:2"))

### Extra: Adicionando e Gerenciando TTL em Subchaves de Hashes

Com o Redis 7.4+, agora é possível adicionar TTL (Time-To-Live) em subchaves de hashes, permitindo que cada campo tenha sua própria expiração. Essa funcionalidade é muito útil para cenários onde os dados de diferentes campos de um hash possuem ciclos de vida distintos.

**Vamos fazer duas ações com o TTL:**
1.	Adicionar um TTL ao campo já existente age.
2.	Adicionar uma nova subchave nickname com um TTL também.

#### Passo 1: Adicionar um TTL ao campo age

Primeiro, adicionamos um TTL de 15 segundos ao campo age no hash do nosso MVP (mvp:2).



In [31]:
# Configurando o TTL do campo 'age' para 15 segundos
r.hexpire("mvp:2", 15, "age")

# Verificando o TTL restante do campo 'age'
ttl_age = r.httl("mvp:2", "age")
print(f"TTL restante do campo 'age': {ttl_age} segundos")

TTL restante do campo 'age': [15] segundos


#### Passo 2: Adicionar uma nova subchave nickname com TTL

Também podemos adicionar uma nova subchave e já configurar seu TTL. Aqui, vamos adicionar a subchave nickname com um valor e expiração de 15 segundos.


In [32]:
# Adicionando o campo 'nickname' com valor e configurando o TTL
r.hset("mvp:2", "nickname", "O Rei")
r.hexpire("mvp:2", 15, "nickname")

# Verificando o TTL restante do campo 'nickname'
ttl_nickname = r.httl("mvp:2", "nickname")
print(f"TTL restante do campo 'nickname': {ttl_nickname} segundos")

TTL restante do campo 'nickname': [15] segundos


Após 15 segundos, o campo nickname também será removido do hash.

**Nota:** Use o Redis Insight para visualizar as mudanças no hash em tempo real. Você verá os campos sendo removidos automaticamente conforme o TTL expira.

# RedisJSON - Estrutura de Dados para Documentos JSON

**RedisJSON** adiciona o tipo de dado JSON ao Redis, permitindo que você armazene e manipule objetos JSON de forma nativa, sem a necessidade de serializar ou desserializar grandes strings JSON no cliente.

Trabalhar com JSON no Redis também permite que você crie um modelo ou schema com campos tipados e metadados. Esses campos suportam buscas avançadas, incluindo consultas multi-facetadas, vetoriais, geográficas e filtradas, todas executadas no lado do servidor. Isso reduz significativamente o tráfego de rede e otimiza o desempenho da aplicação.

> Em outras palavras, você pode consultar e filtrar dados diretamente no Redis, obtendo apenas as informações relevantes para sua aplicação, em uma única ida e volta.

Para a manipulação de dados JSON, podemos usar o Redis CLI (com comandos como `JSON.GET`, `JSON.SET`) ou uma biblioteca cliente, como `redis-py`, com métodos como `json().get()` e `json().set()`. Outro recurso útil é o [RedisOM](https://redis.io/docs/latest/integrate/redisom-for-python/), uma biblioteca de Object Mapping que facilita a manipulação de dados JSON.

- Lista completa de comandos do RedisJSON: [Comandos RedisJSON](https://redis.io/commands/?group=json)
- Documentação Python: [RedisJSON - redis-py](https://redis-py.readthedocs.io/en/stable/redismodules.html#redisjson-commands)

---

## Hands-on: Exercícios com JSON no Redis

### Criar um Objeto JSON

Vamos criar um objeto JSON representando um colégio e salvar este objeto no Redis.

In [ ]:
import redis
from redis.commands.json.path import Path

# Conectar ao Redis
r = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD,
    decode_responses=True
)

# Definir o objeto JSON para um colégio em Curitiba
colegio = {
    "name": "Colégio Positivo",
    "description": "Localizado em Curitiba, o Colégio Positivo é conhecido por seu currículo abrangente e inovador, com foco em tecnologia e excelência acadêmica.",
    "class": "particular",
    "type": ["moderno"],
    "address": {"city": "Curitiba", "street": "Rua Prof. Pedro Viriato Parigot de Souza"},
    "students": 1500,
    "location": "-49.275437,-25.428356",
    "status_log": ["novo", "em operação"],
    "teachers": [
        {"name": "Maria Oliveira", "subjects": ["Biologia", "Química"]},
        {"name": "João Lima", "subjects": ["Matemática", "Física"]}
    ],
    "tags": "excelência acadêmica, inovação, tecnologia"
}

# Salvar o objeto JSON no Redis com a chave 'colegio:42'
r.json().set("colegio:42", '$', colegio)

# Verificar se o objeto foi salvo corretamente
colegio_salvo = r.json().get("colegio:42")
print("Dados do colégio salvos no Redis:", colegio_salvo)

### Exemplos de Leitura no Redis CLI

Abaixo estão alguns exemplos de comandos para leitura no `redis-cli`:

1. **Pegar o JSON inteiro**
   - Retorna todo o objeto JSON armazenado na chave `colegio:42`


In [ ]:
!redis-cli $REDIS_CONN JSON.GET colegio:42

2.	**Pegar propriedades específicas**
  - Retorna apenas os campos `name` e `address.city` do objeto JSON


In [ ]:
!redis-cli $REDIS_CONN JSON.GET colegio:42 $.name $.address.city

3.	**Pegar uma propriedade aninhada**
  - Retorna o nome do primeiro professor na lista de teachers


In [ ]:
!redis-cli $REDIS_CONN JSON.GET colegio:42 $.teachers[0].name

4.	**Pegar o primeiro elemento de um array**
  - Retorna o primeiro status do status_log

In [ ]:
!redis-cli $REDIS_CONN JSON.GET colegio:42 $.status_log[0]

### Escrita e Manipulação de JSON no Redis

Além de criar e ler, o Redis permite atualizar partes específicas do **JSON**, sem a necessidade de substituir o documento inteiro. Abaixo estão alguns exemplos para modificar o JSON.

In [ ]:
# Imprimir o estado inicial do array status_log e do campo students
status_inicial = r.json().get("colegio:42", Path("$.status_log"))
alunos_iniciais = r.json().get("colegio:42", Path("$.students"))
print("Status inicial:", status_inicial)
print("Número de alunos inicial:", alunos_iniciais)

print("----------------------------------------------------------------")

# 1. Adicionar um novo status ao array status_log
r.json().arrappend("colegio:42", Path("$.status_log"), "em expansão")
status_atualizado = r.json().get("colegio:42", Path("$.status_log"))
print("Status atualizado:", status_atualizado)

# 2. Incrementar o número de alunos em 10
r.json().numincrby("colegio:42", Path("$.students"), 10)
alunos_atualizados = r.json().get("colegio:42", Path("$.students"))
print("Número de alunos atualizado:", alunos_atualizados)

# Search and Query: Explorando Consultas Avançadas no Redis


Agora que temos os dados salvos como objetos JSON, vamos usar o **RediSearch** para criar um índice e realizar buscas avançadas. RediSearch permite declarar índices com facilidade e automatiza a adição de novos objetos ao índice, essencialmente transformando o Redis em um banco de dados NoSQL para documentos.

Primeiro, vamos carregar dados de várias escolas conhecidas no Brasil. Esses dados servirão para testar as consultas avançadas.

In [ ]:
schools = [
    {
        "name": "Colégio Bandeirantes",
        "description": "Abrangendo 10 estados, o currículo premiado desta escola inclui um sistema de leitura abrangente (do reconhecimento de letras e fonética à leitura de livros completos), além de matemática, ciências, estudos sociais e até filosofia.",
        "class": "independente",
        "type": ["tradicional"],
        "address": {"city": "São Paulo", "street": "Rua Estela"},
        "students": 342,
        "location": "-46.633308,-23.550520",
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Ana Silva",
                "subjects": ["Matemática", "Física"]
            },
            {
                "name": "Carlos Souza",
                "subjects": ["História", "Geografia"]
            }
        ],
        "tags": "excelência acadêmica, tecnologia, inovação, java"
    },
    {
        "name": "Escola Maria Imaculada",
        "description": "A Garden School é uma nova e inovadora experiência de ensino e aprendizado ao ar livre, oferecendo atividades ricas e variadas em um ambiente natural para crianças e famílias.",
        "class": "estadual",
        "type": ["floresta", "montessori"],
        "address": {"city": "São Paulo", "street": "Avenida Vicente Rao"},
        "students": 1452,
        "location": "-46.699687,-23.621993",
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Mariana Costa",
                "subjects": ["Biologia", "Química"]
            },
            {
                "name": "Rafael Lima",
                "subjects": ["Português", "Literatura"]
            }
        ],
        "tags": "educação ao ar livre, inovação, sustentabilidade"
    },
    {
        "name": "Colégio São Luís",
        "description": "A Gillford School é um centro de aprendizado inclusivo que acolhe pessoas de todos os estilos de vida, convidando-as a assumir seu papel como agentes regenerativos, criando novos caminhos para o futuro e incitando um movimento internacional de transformação cultural, territorial e social.",
        "class": "privada",
        "type": ["democrática", "waldorf"],
        "address": {"city": "Rio de Janeiro", "street": "Rua Haddock Lobo"},
        "students": 721,
        "location": "-46.660213,-23.558704",
        "status_log": ["novo", "em operação", "fechada"],
        "teachers": [
            {
                "name": "Beatriz Mendes",
                "subjects": ["Filosofia", "Sociologia"]
            },
            {
                "name": "Fernando Almeida",
                "subjects": ["Artes", "Educação Física"]
            }
        ],
        "tags": "inclusão, transformação social, interdisciplinaridade"
    },
    {
        "name": "Escola Sesc de Ensino",
        "description": "A filosofia por trás da Forest School baseia-se no desejo de proporcionar às crianças pequenas uma educação que incentive a apreciação do mundo amplo na natureza, enquanto alcançam independência, confiança e alta autoestima.",
        "class": "independente",
        "type": ["floresta", "montessori", "democrática"],
        "address": {"city": "Rio de Janeiro", "street": "Rua Jacarepaguá"},
        "students": 1000,
        "location": "-43.375162,-22.972250",
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Luciana Oliveira",
                "subjects": ["Ciências", "Matemática"]
            },
            {
                "name": "Paulo Santos",
                "subjects": ["História", "Geografia"]
            }
        ],
        "tags": "natureza, independência, confiança"
    },
    {
        "name": "Colégio Móbile - Perfeito",
        "description": "A Escola Móbile oferece um programa acadêmico rigoroso combinado com foco em pensamento crítico e criatividade, preparando os alunos para o ensino superior e além.",
        "class": "privada",
        "type": ["tradicional"],
        "address": {"city": "Curitiba", "street": "Rua Cotovia"},
        "students": 600,
        "location": "-46.676201,-23.592920",
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Juliana Pereira",
                "subjects": ["Química", "Biologia"]
            },
            {
                "name": "Ricardo Fernandes",
                "subjects": ["Português", "Inglês"]
            }
        ],
        "tags": "rigor acadêmico, pensamento crítico, criatividade"
    },
    {
        "name": "Colégio Santa Cruz",
        "description": "O Colégio Santa Cruz é conhecido por seu forte desempenho acadêmico e compromisso com a responsabilidade social, promovendo um senso de comunidade entre os alunos.",
        "class": "privada",
        "type": ["tradicional", "religiosa"],
        "address": {"city": "Belo Horizonte", "street": "Rua Orobó"},
        "students": 300,
        "location": "-46.712750,-23.547680",
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Gabriela Rocha",
                "subjects": ["Religião", "Filosofia"]
            },
            {
                "name": "Eduardo Matos",
                "subjects": ["Educação Física", "Ciências"]
            }
        ],
        "tags": "desempenho acadêmico, responsabilidade social, comunidade"
    },
    {
        "name": "Colégio Rio Branco",
        "description": "O Colégio Rio Branco oferece um currículo diversificado com forte ênfase em línguas, artes e ciências, preparando os alunos para a cidadania global.",
        "class": "privada",
        "type": ["internacional"],
        "address": {"city": "São Paulo", "street": "Rua Alves Guimarães"},
        "students": 50,
        "location": "-46.670912,-23.555451",
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Marcelo Andrade",
                "subjects": ["Inglês", "Espanhol"]
            },
            {
                "name": "Sofia Ribeiro",
                "subjects": ["Artes", "História"]
            }
        ],
        "tags": "línguas, artes, cidadania global, java"
    }
]



# Carregar os dados no Redis como objetos JSON
for id, school in enumerate(schools):
    r.json().set(f"school_json:{id}", '.', school)

print("Dados das escolas carregados com sucesso!")

## Visualizando Dados com redis-cli

Aqui estão alguns exemplos de como você pode visualizar os dados diretamente no redis-cli:



In [ ]:
# Recuperar o JSON completo de uma escola específica
!redis-cli $REDIS_CONN JSON.GET school_json:1 $

# Listar as chaves de todas as escolas carregadas
!redis-cli $REDIS_CONN keys 'school_json:*'

Para fazer o mesmo em Python:

In [ ]:
# Recuperar o JSON completo usando Python
res = r.json().get("school_json:0", "$")
print(res)

## Configurando o Índice no Redis com RediSearch

Vamos configurar um índice para o RediSearch. Este índice incluirá campos como nome, descrição, cidade, número de alunos e tags. Ele também suportará geolocalização, permitindo consultas espaciais.



In [ ]:
from redis.commands.search.field import TextField, TagField, NumericField, GeoField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType

# Excluir o índice, caso já exista, para evitar conflitos
try:
    r.ft("idx:schools_json").dropindex(delete_documents=False)
except:
    pass

# Criar o índice para os dados de escola
schema = (
    TextField("$.name", as_name="name"),
    TextField("$.description", as_name="description"),
    TagField("$.address.city", as_name="city"),
    NumericField("$.students", as_name="students"),
    TagField("$.tags", as_name="tags", separator=","),
    GeoField("$.location", as_name="location")
)

# Configurar o índice
r.ft("idx:schools_json").create_index(
    schema,
    definition=IndexDefinition(prefix=["school_json:"], index_type=IndexType.JSON)
)
print("Índice criado com sucesso!")

## Testando o Índice com uma Full-Text Search



Vamos fazer uma busca completa (full-text) pelo termo “rigoroso” para verificar se o índice está funcionando corretamente. A busca retornará o documento que contém o termo na descrição.

In [ ]:
import pandas as pd

# Executar uma Full-Text Search pelo termo "rigoroso"
res = r.ft("idx:schools_json").search("rigoroso")
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print("Resultado da Full-Text Search:")
display(res_df)

### Recuperando Campos Específicos

Para otimizar o tráfego de dados, você pode escolher retornar apenas alguns campos.

In [ ]:
from redis.commands.search.query import Query

# Busca pelo termo "rigoroso", retornando apenas a cidade e o nome
query = Query("rigoroso") \
   .return_field("$.address.city", as_field="city") \
   .return_field("$.name", as_field="name")
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
display(res_df)

### Exemplo Avançado: Busca Multi-Faceta e por Intervalo

Agora, vamos fazer uma consulta um pouco mais complexa. Suponha que queremos encontrar escolas em São Paulo com um número de alunos entre 300 e 2000. Esse exemplo usa um match exato (TAG) e um intervalo numérico.


In [ ]:
# Busca de escolas em São Paulo com 300 a 2000 alunos
query = Query('@city:{São Paulo} @students:[300, 2000]') \
   .return_field("$.address.city", as_field="city") \
   .return_field("$.name", as_field="name") \
   .return_field("$.students", as_field="students")
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
display(res_df)

Esses exemplos mostram como o Redis pode ser utilizado para criar consultas avançadas em dados JSON, oferecendo uma abordagem eficiente e escalável para buscas complexas e multi-dimensionais.

Essa estrutura mantém o foco em introduzir a criação de índices e execução de consultas com `RediSearch`, cobrindo tanto buscas simples quanto avanços para consultas de múltiplas facetas e intervalos.

# Casos de uso poderosos e mais avançados
*(não são complexos, é tranquilo entender)*

## Busca por Tags - Encontrando escolas com características específicas

Nesta seção, vamos realizar buscas utilizando tags.

Tags permitem buscas exatas em campos específicos. Mas isso não quer dizer que você não pode usar wildcards.


In [ ]:
# Busca por tags com suporte a wildcards - PT-BR
query = Query('@tags:{inovaçã*}') \
    .return_field("$.address.city", as_field="city") \
    .return_field("$.name", as_field="name")
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print(res_df)

## Busca por Intervalo - Quantos alunos estão matriculados?

Para realizar buscas por intervalos em campos numéricos, como a quantidade de alunos em uma escola, podemos utilizar consultas de intervalo.

**Não se assustem com `inf+` e `inf-`!**
Esses termos são utilizados para representar o intervalo máximo e mínimo.\
Eles são úteis quando você quer incluir todos os valores acima ou abaixo de um determinado limite.

In [ ]:
# Busca por escolas com entre 300 e 1000 alunos (inclusivo)
search = '@students:[300 1000]'
query = Query(search) \
    .return_field("$.address.city", as_field="city") \
    .return_field("$.name", as_field="name") \
    .return_field("$.students", as_field="students")
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print("--------------------------------------------------------------------")
print("Escolas com entre 300 e 1000 estudantes")
print(res_df)

# Busca por escolas com mais de 1000 alunos (exclusivo)
search = '@students:[(1000 +inf]'
query = Query(search) \
    .return_field("$.address.city", as_field="city") \
    .return_field("$.name", as_field="name") \
    .return_field("$.students", as_field="students")
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print("--------------------------------------------------------------------")
print("Escolas com mais de 1000 estudantes")
print(res_df)

# Busca por escolas com menos de 300 alunos (inclusivo)
search = '@students:[-inf 300]'
query = Query(search) \
    .return_field("$.address.city", as_field="city") \
    .return_field("$.name", as_field="name") \
    .return_field("$.students", as_field="students")
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print("--------------------------------------------------------------------")
print("Escolas com menos de 300 estudantes")
print(res_df)

## Fuzzy Match - Busca com Correspondência Aproximada

Para realizar buscas onde o termo exato pode não ser conhecido, o Redis permite a correspondência aproximada (fuzzy match) utilizando a distância de Levenshtein.

In [ ]:
# Busca aproximada pelo nome "Bondeirants", permitindo erro de digitação
query = Query('%%Bondeirants%%') \
   .return_field("$.address.city", as_field="city") \
   .return_field("$.name", as_field="name") \
   .return_field("$.students", as_field="students")
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print("Escolas encontradas com fuzzy match para 'Bondeirants':")
display(res_df)

## Suporte a Sinônimos - Agrupando Termos Relacionados

O Redis Stack permite a criação de grupos de sinônimos, possibilitando buscas que retornam documentos contendo termos semelhantes.



In [ ]:
# Configurar grupo de sinônimos (exemplo não usual para ilustrar a flexibilidade)
r.ft("idx:schools_json").synupdate("synonym_maluco_1", False, "Perfeito", "CristianoRonaldo", "CR7")

# Buscar utilizando sinônimo
search = 'CristianoRonaldo'
query = Query(search) \
    .return_field("$.address.city", as_field="city") \
    .return_field("$.name", as_field="name") \
    .return_field("$.description", as_field="description")
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print("Busca por 'Cristiano Ronaldo' usando sinônimos:")
display(res_df)

## Suporte a Stemming - Buscando pelo Radical da Palavra

O **Redis Stack** oferece suporte a Stemming, permitindo que a busca por um termo inclua outras variações com o mesmo radical.

In [ ]:
# Criação do índice com suporte a Stemming em inglês
try:
    r.ft("idx:schools_stemming").dropindex(delete_documents=False)
except:
    pass

schema = (
    TextField("$.name", as_name="name"),
    TextField("$.description", as_name="description")
)
r.ft("idx:schools_stemming").create_index(schema, definition=IndexDefinition(prefix=["school_json:"], index_type=IndexType.JSON, language='english'))

# Adicionar documentos de exemplo para teste de stemming
r.json().set('school_json:10', '$', {"name": "School Hiring", "description": "just to avoid confusion"})
r.json().set('school_json:11', '$', {"name": "College Hire", "description": "just to avoid confusion"})
r.json().set('school_json:12', '$', {"name": "Academy of Hired", "description": "just to avoid confusion"})

# Busca com Stemming para "hiring"
search = '@name:(hiring)'
query = Query(search) \
    .return_field("$.name", as_field="name") \
    .return_field("$.description", as_field="description")
res = r.ft("idx:schools_stemming").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print("Busca por 'hiring' com suporte a Stemming:")
display(res_df)

## Suporte a Correspondência Fonética - Buscas por Similaridade de Som

A correspondência fonética no Redis usa o algoritmo Double Metaphone (DM) para encontrar termos similares em som, como variações de nomes próprios.

In [ ]:
# Criar o índice com correspondência fonética
!redis-cli $REDIS_CONN FT.DROPINDEX idx:schools_phonetic || true
!redis-cli $REDIS_CONN FT.CREATE idx:schools_phonetic ON JSON PREFIX 1 school_json: LANGUAGE portuguese SCHEMA $.name AS name TEXT PHONETIC dm:pt $.description AS description TEXT

# Adicionar documentos para busca fonética
!redis-cli $REDIS_CONN JSON.SET school_json:30 $ "{\"name\": \"Escola João\", \"description\": \"Um ótimo lugar para aprender\"}"
!redis-cli $REDIS_CONN JSON.SET school_json:31 $ "{\"name\": \"Escola Jão\", \"description\": \"Uma excelente instituição educacional\"}"
!redis-cli $REDIS_CONN JSON.SET school_json:32 $ "{\"name\": \"Escola Joãozinho\", \"description\": \"Educação de primeira qualidade\"}"

# Executar consultas fonéticas
!echo "Busca por 'João' que deve retornar documentos com 'João' e 'Jão', em ordem decrescente"
!redis-cli $REDIS_CONN FT.SEARCH idx:schools_phonetic "João" LANGUAGE portuguese SORTBY name DESC RETURN 1 name

!echo "Busca por 'Jão' que deve retornar documentos com 'Jão' e 'João', em ordem crescente"
!redis-cli $REDIS_CONN FT.SEARCH idx:schools_phonetic "Jão" LANGUAGE portuguese SORTBY name ASC RETURN 1 name

!echo "Busca por 'Joãozinho' que deve retornar apenas o documento exato"
!redis-cli $REDIS_CONN FT.SEARCH idx:schools_phonetic "Joãozinho" LANGUAGE portuguese SORTBY name DESC

Essas consultas demonstram a flexibilidade do Redis em suportar buscas complexas e avançadas, como correspondências fonéticas, aproximações e agrupamentos por sinônimos. Cada exemplo pode ser adaptado para casos de uso específicos no desenvolvimento de aplicações robustas e ricas em recursos de pesquisa.

# Busca Complexa no Redis: O Grand Finale

Nesta última parte do nosso workshop, vamos realizar uma busca complexa combinando várias funcionalidades avançadas. Vamos explorar:

- **Fuzzy Search:** Para encontrar termos similares ao termo de busca.
- **Full Text Search:** Para buscar termos em campos de texto.
- **Tag Search:** Para encontrar correspondências exatas em campos de tags.
- **Geolocation Search:** Para buscar documentos dentro de um raio específico.
- **Range Search:** Para buscar documentos com valores numéricos em intervalos.
- **Sorting:** Para ordenar os resultados.

In [ ]:
# Busca Complexa com Fuzzy, Full Text, Tag, Geolocation, Range e Ordenação
search = '%%Colejio%% (@name:Colégio|Banana) @city:{São Paulo} @tags:{tecnologia|ambiental|java} @location:[-46.633308 -23.550520 100000 km] @students:[25 400]'
query = Query(search) \
    .return_field("$.name", as_field="name") \
    .return_field("$.students", as_field="students") \
    .sort_by("students", asc=False)
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])

print("--------------------------------------------------------------------")
print("Busca Complexa: Fuzzy, Full Text, Tag, Geoloc, Range e Sort")
print("Search: {0}".format(search))
display(res_df)
print("--------------------------------------------------------------------")

## Agregações - Agrupando e Somando Resultados com FT.AGGREGATE

As agregações permitem criar relatórios analíticos ou consultas facetadas no Redis. Vamos agrupar escolas por cidade e contar o número de escolas, e depois fazer o mesmo com a soma de estudantes.

In [ ]:
from redis.commands.search.aggregation import AggregateRequest
from redis.commands.search import reducers

# Helper para exibir resultados do FT.AGGREGATE como DataFrame
def display_ft_agg(res):
    data = [[item for item in sublist] for sublist in res.rows]
    columns = {sublist[i]: [] for sublist in data for i in range(0, len(sublist), 2)}
    for sublist in data:
        for i in range(0, len(sublist), 2):
            columns[sublist[i]].append(sublist[i + 1])
    df = pd.DataFrame(columns)
    display(df)

# Agrupamento por cidade para contar o número de escolas
request = AggregateRequest(f'*').group_by('@city', reducers.count().alias('count'))
res = r.ft("idx:schools_json").aggregate(request)
print("Número de escolas por cidade:")
display_ft_agg(res)

# Agrupamento por cidade para somar o número de estudantes
request = AggregateRequest(f'*').group_by('@city', reducers.sum('@students').alias('students_count'))
res = r.ft("idx:schools_json").aggregate(request)
print("Número total de estudantes por cidade:")
display_ft_agg(res)

## Bonus: Verificação Ortográfica com FT.SPELLCHECK

A verificação ortográfica permite corrigir termos com erros de digitação ou grafia, sugerindo alternativas com base na distância de Levenshtein.

### Configuração do Índice para Spellcheck

Primeiro, criamos um índice e adicionamos alguns documentos.



In [ ]:
# Criar o índice para Spellcheck
!redis-cli $REDIS_CONN FT.DROPINDEX idx:spellcheck || true
!redis-cli $REDIS_CONN FT.CREATE idx:spellcheck ON JSON PREFIX 1 doc: SCHEMA $.content AS content TEXT

# Adicionar documentos de exemplo
!redis-cli $REDIS_CONN JSON.SET doc:1 $ "{\"content\": \"Redis is an in-memory data structure store.\"}"
!redis-cli $REDIS_CONN JSON.SET doc:2 $ "{\"content\": \"Redis provides high availability through Redis Sentinel.\"}"
!redis-cli $REDIS_CONN JSON.SET doc:3 $ "{\"content\": \"Redis is used for caching, real-time analytics, and message brokering.\"}"

### Execução do Spellcheck

Agora vamos testar a correção ortográfica com diferentes níveis de Levenshtein.

In [ ]:
# Correção ortográfica para "Redsi" com DISTANCE 1
!echo "Verificação ortográfica para 'Redsi' com DISTANCE 1:"
!redis-cli $REDIS_CONN FT.SPELLCHECK idx:spellcheck Redsi DISTANCE 1
!echo ""

# Correção ortográfica para "Rédiz" com DISTANCE 1
!echo "Verificação ortográfica para 'Rédiz' com DISTANCE 1:"
!redis-cli $REDIS_CONN FT.SPELLCHECK idx:spellcheck Rédiz DISTANCE 1
!echo ""

# Correção ortográfica para "Rédiz" com DISTANCE 2
!echo "Verificação ortográfica para 'Rédiz' com DISTANCE 2:"
!redis-cli $REDIS_CONN FT.SPELLCHECK idx:spellcheck Rédiz DISTANCE 2
!echo ""

Com essa última seção, você explorou alguns dos recursos mais avançados e poderosos do Redis, combinando buscas complexas, agregações, e verificações ortográficas. Pronto para o próximo nível: busca com vetores e integração com modelos de Machine Learning no próximo notebook!